In [10]:
import os
prefix = "/home/kreffert/"
prefix = "/pfs/data6/home/ma/ma_ma/ma_kreffert/" 
os.chdir(f'{prefix}Probabilistic_LTSF/BasicTS/')
from basicts.metrics import masked_mae, masked_mse, nll_loss, crps, Evaluator, quantile_loss, empirical_crps
from easytorch.device import set_device_type
from easytorch.utils import get_logger, set_visible_devices
# set the device type (CPU, GPU, or MLU)
device_type ='gpu'
gpus = '0'
set_device_type(device_type)
set_visible_devices(gpus)
from easydict import EasyDict
from tqdm import tqdm

# extract the paths to the configs and weights
import yaml
# /home/kreffert/Probabilistic_LTSF/notebooks/Final plots/weights.yaml
with open(f'{prefix}Probabilistic_LTSF/notebooks/Final plots/weights.yaml', 'r') as file:
    _configs = yaml.safe_load(file)
import glob 
def reconstruct_paths(
    simplified_dict,
    _dataset=['ETTh1', 'ETTm1'],
    _models=['DLinear', 'PatchTST', 'DeepAR'],
    _dists=['q', 'iq', 'u', 'm'],
    _seeds=[0, 1, 2, 3, 4],
    _model_dist_map=None  # optional: {'DLinear': ['m'], 'DeepAR': ['u']}
):
    base_path = "final_weights/"
    dist_mapping = {"iq": "i_quantile", "u": "univariate", "m": "multivariate", "q": "quantile"}
    filtered_dict = {}

    for dataset, models in simplified_dict.items():
        if dataset not in _dataset:
            continue
        for model, dists in models.items():
            if _model_dist_map:
                if model not in _model_dist_map:
                    continue
            elif model not in _models:
                continue

            allowed_dists = _model_dist_map[model] if _model_dist_map else _dists

            for dist, seeds in dists.items():
                if dist not in allowed_dists:
                    continue
                _cfg = f"{dataset}_prob_quantile.py" if dist in ["q", "iq"] else f"{dataset}_prob.py"
                _ckpt = "_best_val_QL.pt" if dist in ["q", "iq"] else "_best_val_NLL.pt"
                mapped_dist = dist_mapping.get(dist, dist)

                for seed, path_suffix in seeds.items():
                    if seed not in _seeds or path_suffix is None:
                        continue
                    prefix = f"{base_path}{dataset}/{model}/{mapped_dist}/{seed}/{path_suffix}"
                    log_pattern = os.path.join(prefix, "training_log_*.log")
                    log_files = glob.glob(log_pattern)
                    log_file = log_files[-1] if log_files else None
                    filtered_dict.setdefault(dataset, {}).setdefault(model, {}).setdefault(dist, {})[seed] = {
                        'cfg': f"{prefix}/{_cfg}",
                        'ckpt': f"{prefix}/{model}{_ckpt}",
                        'log': log_file,
                    }

    return filtered_dict



_model_dist_map={'DLinear': ['m'], 'DeepAR': ['u']}
_models = ['DeepAR'] #full_dict[dataset][model][dist][random_state]
_dataset = ['ETTh1']
_dists = ['u', 'm']
_seeds = [0, 1, 2]
_configs = reconstruct_paths(_configs, _dataset=_dataset, _models=_models, _dists=_dists, _seeds=_seeds, _model_dist_map=_model_dist_map)

2025-07-01 10:08:57,578 - easytorch-env - INFO - Use devices 0.


In [21]:
import re

def parse_log_for_epochs(log_path):
    epoch_pattern = re.compile(r"Epoch (\d+) / (\d+)")
    train_time_pattern = re.compile(r"train/time: ([\d.]+) \(s\)")
    params_pattern = re.compile(r"Number of parameters:\s*(\d+)")

    num_parameters = None
    epochs = []
    durations = []

    with open(log_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        # Extract number of parameters (only once)
        if num_parameters is None:
            params_match = params_pattern.search(line)
            if params_match:
                num_parameters = int(params_match.group(1))

        epoch_match = epoch_pattern.search(line)
        if epoch_match:
            epoch_num = int(epoch_match.group(1))
            epochs.append(epoch_num)

        time_match = train_time_pattern.search(line)
        if time_match:
            durations.append(float(time_match.group(1)))

    num_epochs = len(epochs)
    return num_epochs, durations, num_parameters

def extract_data(simplified_dict):
    epoch_dict = {}
    for dataset, models in simplified_dict.items():
        for model, dists in models.items():
            for dist, seeds in dists.items():
                _cfg = f"{dataset}_prob_quantile.py" if dist in ["q", "iq"] else f"{dataset}_prob.py"
                _ckpt = "_best_val_QL.pt" if dist in ["q", "iq"] else "_best_val_NLL.pt"
                for seed, path_suffix in seeds.items():
                    log_path = simplified_dict[dataset][model][dist][seed]['log']
                    if log_path is not None:
                        num_epochs, epoch_durations, num_parameters = parse_log_for_epochs(log_path)
                        epoch_dict.setdefault(dataset, {}).setdefault(model, {}).setdefault(dist, {})[seed] = {
                            'num_epochs': num_epochs,
                            'epoch_durations': epoch_durations,
                            'num_parameters': num_parameters
                        }

    return epoch_dict

import pprint
pp = pprint.PrettyPrinter(indent=4, width=200)
epoch_dict = extract_data(_configs)
pp.pprint(epoch_dict)

{   'ETTh1': {   'DLinear': {   'm': {   2: {   'epoch_durations': [1085.83, 1082.46, 1081.83, 1080.72, 1077.76, 1082.34, 1082.08, 1087.5, 1082.16, 1082.08, 1082.41],
                                                'num_epochs': 11,
                                                'num_parameters': 188061120}}},
                 'DeepAR': {   'u': {   0: {   'epoch_durations': [183.74, 181.54, 181.25, 181.28, 181.04, 181.13, 180.99, 180.95, 180.45, 180.79, 180.56, 181.25, 179.9, 179.4],
                                               'num_epochs': 14,
                                               'num_parameters': 81307},
                                        2: {'epoch_durations': [212.25, 208.94, 208.77, 208.31, 208.74, 208.89, 208.81, 209.19, 208.55, 208.91, 209.04], 'num_epochs': 11, 'num_parameters': 81307}}}}}


In [18]:
import numpy as np

def compute_stats_across_seeds(epoch_dict):
    stats_dict = {}

    for dataset, models in epoch_dict.items():
        stats_dict[dataset] = {}
        for model, dists in models.items():
            stats_dict[dataset][model] = {}
            for dist, seeds in dists.items():
                num_epochs_list = []
                avg_duration_per_seed = []
                num_parameters_list = []

                for seed, data in seeds.items():
                    num_epochs_list.append(data['num_epochs'])
                    # Average epoch duration within this seed
                    avg_duration = np.mean(data['epoch_durations'])
                    avg_duration_per_seed.append(avg_duration)
                    num_parameters_list.append(data['num_parameters'])

                avg_epochs = np.mean(num_epochs_list)
                std_epochs = np.std(num_epochs_list)

                avg_durations = np.mean(avg_duration_per_seed)
                std_durations = np.std(avg_duration_per_seed)

                avg_params = np.mean(num_parameters_list)
                std_params = np.std(num_parameters_list)

                stats_dict[dataset][model][dist] = {
                    'avg_num_epochs': avg_epochs,
                    'std_num_epochs': std_epochs,
                    'avg_epoch_duration': avg_durations,
                    'std_epoch_duration': std_durations,
                    'avg_num_parameters': avg_params,
                    'std_num_parameters': std_params,
                }

    return stats_dict

stats = compute_stats_across_seeds(epoch_dict)
pp.pprint(stats)

{ 'ETTh1': { 'DLinear': { 'm': { 'avg_epoch_duration': np.float64(1082.47),
                                 'avg_num_epochs': np.float64(11.0),
                                 'avg_num_parameters': np.float64(188061120.0),
                                 'std_epoch_duration': np.float64(0.0),
                                 'std_num_epochs': np.float64(0.0),
                                 'std_num_parameters': np.float64(0.0)}},
             'DeepAR': { 'u': { 'avg_epoch_duration': np.float64(195.07327922077923),
                                'avg_num_epochs': np.float64(12.5),
                                'avg_num_parameters': np.float64(81307.0),
                                'std_epoch_duration': np.float64(14.053993506493512),
                                'std_num_epochs': np.float64(1.5),
                                'std_num_parameters': np.float64(0.0)}}}}
